# 变分自编码器

变分自编码器（Variational Auto-Encoder, VAE）是一种生成模型，结合了概率图模型和神经网络的优势。它通过引入潜在变量来建模数据的生成过程，并使生成过程中的神经网络可计算化，从而实现高效的训练和生成。在传统的**自编码器（Autoencoder）**中，模型将输入图像压缩成一个固定的向量（编码），再还原回来。但这种方式有个问题：潜空间（Latent Space）是不连续的。随便选一个随机向量，还原出来的往往是乱码。VAE 的改进之处在于： 它不把输入映射为一个孤立的点，而是映射为一个概率分布（通常是高斯分布）。
- 传统 AE： 输入一张猫 $\rightarrow$ 编码为数值 [0.5, -1.2]。
- VAE： 输入一张猫 $\rightarrow$ 编码为一个均值 $\mu$ 和方差 $\sigma$ $\rightarrow$ 意味着这张猫在潜空间里是一个“区域”。

## 原理

生成模型的核心思想是计算$p(x)$。 但直接计算$p(x)$往往很困难，于是引入潜变量$z$，将$p(x)$表示为对潜变量的积分：
\begin{equation}
\label{eq:vae-marginal}
p(x) = \int p(x|z)p(z)dz
\end{equation}
但在高维空间，对所有可能的$z$进行积分是不可行的。由于不能直接计算$p(x)$，也就无法直接通过$p(z|x)=p(x|z)p(z)/p(x)$来计算后验分布。

为了解决这个问题，VAE引入一个容易处理的分布$q(z|x)$来近似$p(z|x)$。这个分布通常称为“变分分布”（variational distribution）。我们的目标是让这两个分布尽可能的接近，即最小化它们之间的Kullback-Leibler散度（KL散度）：
\begin{equation}
\label{eq:vae-kl}
KL(q(z|x) || p(z|x)) = \int q(z|x) \log \frac{q(z|x)}{p(z|x)} dz
\end{equation}
将$p(z|x) = \frac{p(x,z)}{p(x)}$代入上式，并利用对数的性质，可以将KL散度重写为：
\begin{equation}
\label{eq:vae-kl-expanded}
KL(q(z|x) || p(z|x)) = \int q(z|x) \log \frac{q(z|x)}{p(x,z)} dz + \log p(x)
\end{equation}
注意到$\log p(x)$与$z$无关，可以将其移出积分。重新整理上式，可以得到：
\begin{equation}
\label{eq:vae-evidence-lower-bound}
\log p(x) = KL(q(z|x) || p(z|x)) + \int q(z|x) \log \frac{p(x,z)}{q(z|x)} dz
\end{equation}
通过上式可以看出，$\log p(x)$等于KL散度加上一个积分项。由于KL散度总是非负的，我们可以得到以下不等式：
\begin{equation}
\label{eq:vae-elbo}
\log p(x) \geq \int q(z|x) \log \frac{p(x,z)}{q(z|x)} dz
\end{equation}
这个积分项被称为“证据下界”（Evidence Lower Bound, ELBO）。最大化ELBO等价于最小化KL散度，从而使$q(z|x)$更接近$p(z|x)$。

ELBO可以进一步分解为两部分：    
\begin{equation}
\label{eq:vae-elbo-decomposed}
\int q(z|x) \log \frac{p(x,z)}{q(z|x)} dz = \int q(z|x) \log p(x|z) dz - KL(q(z|x) || p(z))
\end{equation}
其中，
- 第一项表示重构误差，衡量从潜变量$z$生成数据$x$的能力；
- 第二项是正则化项，确保变分分布$q(z|x)$与先验分布$p(z)$接近。

## 神经网络实现

VAE使用神经网络来参数化变分分布$q(z|x)$和生成分布$p(x|z)$。具体来说：
- **编码器**网络：将输入数据$x$映射到潜变量$z$。

在数学推导中，我们假设后验分布$q_{\phi}(z|x)$为多元高斯分布，为了描述这个分布，我们需要学习其均值向量$\mu_{\phi}(x)$和对角协方差矩阵的对角线元素$\sigma_{\phi}(x)$。因此，编码器网络的输出通常会生成两个全连接层，分别对应均值向量和对数方差向量$\log\sigma^2$（为了确保方差 $\sigma^2$ 始终为正数，模型通常预测 $\log \sigma_{\phi}(x)^2$，然后再通过 $e^{\frac{1}{2}\log\sigma^2}$ 计算出标准差 $\sigma$）。

- **解码器**网络：将潜变量$z$映射回数据空间，生成数据$x$。

- 为了使得梯度能够通过采样过程传递，我们使用“**重参数化技巧**”（reparameterization trick），将采样过程表示为一个可微分的操作。  具体来说，我们将潜变量$z$表示为：

\begin{equation}
\label{eq:vae-reparameterization}
z = \mu(x) + \sigma(x) \odot \epsilon
\end{equation}

其中，$\mu(x)$和$\sigma(x)$是编码器网络输出的均值和标准差，$\epsilon$是从标准正态分布采样的噪声，$\odot$表示逐元素乘法。训练过程中，我们通过最大化ELBO来优化编码器和解码器的参数。通过这种方式，我们可以在训练过程中对编码器和解码器进行端到端的优化，从而实现高效的生成模型。


